In [8]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFE, VarianceThreshold
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler


from src.evalution import evaluatelog_result
from src.logger import Logger

In [9]:
logger = Logger(log_filename="metrics.txt")

UNIQUE_LABELS = [0, 1, 2, 3]
FEATURES = ['project_name', 'project_version', 'label', 'code', 'code_comment',
            'code_no_comment', 'lc', 'pi', 'ma', 'nbd', 'ml', 'd', 'mi', 'fo', 'r',
            'e']
TRAIN_COLS = ['lc', 'pi', 'ma', 'nbd', 'ml', 'd', 'mi', 'fo', 'r', 'e']

In [10]:
train = pd.read_csv("../../datasets/data/train.csv")
x_train = train[TRAIN_COLS]
y_train = train["label"]

In [11]:
test = pd.read_csv("../../datasets/data/test.csv")
x_test = test[TRAIN_COLS]
y_test = test["label"]

In [12]:
max_features = min(x_train.shape[1], 9)


def correlation_feature_selection(X, y, threshold=0.8):
    corr_matrix = np.corrcoef(X, rowvar=False)
    selected_features = []
    for i in range(len(corr_matrix)):
        if not any(abs(corr_matrix[i][j]) > threshold for j in selected_features):
            selected_features.append(i)
    return np.array(selected_features[:max_features])

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "MaxAbsScaler": MaxAbsScaler(),
    "RobustScaler": RobustScaler()
}

feature_selectors = {
    "SelectKBest_f_classif": SelectKBest(score_func=f_classif, k=max_features),
    "SelectKBest_mutual_info": SelectKBest(score_func=mutual_info_classif, k=max_features),
    "TreeBased_RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "RFE_LogisticRegression": RFE(estimator=LogisticRegression(max_iter=1000, random_state=42),
                                  n_features_to_select=max_features),
    "VarianceThreshold": VarianceThreshold(threshold=0.01),
    "L1Based_Logistic": SelectFromModel(LinearSVC(C=0.01, penalty='l1', dual=False, max_iter=1000, random_state=42)),
    "CorrelationBased": correlation_feature_selection
}

classifiers = {
    "RandomForest": RandomForestClassifier(),
}


In [14]:
from sklearn.preprocessing import PolynomialFeatures

results = []

for scaler_name, scaler in scalers.items():
    # Veriyi ölçeklendir
    X_train_scaled = scaler.fit_transform(x_train)
    X_test_scaled = scaler.transform(x_test)

    for selector_name, selector in feature_selectors.items():
        if isinstance(selector, (SelectKBest, RFE, VarianceThreshold, SelectFromModel)):
            X_train_selected = selector.fit_transform(X_train_scaled, y_train)
            X_test_selected = selector.transform(X_test_scaled)
        elif isinstance(selector, RandomForestClassifier):
            selector.fit(X_train_scaled, y_train)
            feature_importances = selector.feature_importances_
            top_features = np.argsort(feature_importances)[-max_features:]
            X_train_selected = X_train_scaled[:, top_features]
            X_test_selected = X_test_scaled[:, top_features]
        elif selector_name == "CorrelationBased":
            selected_features = selector(X_train_scaled, y_train)
            X_train_selected = X_train_scaled[:, selected_features]
            X_test_selected = X_test_scaled[:, selected_features]

        for clf_name, clf in classifiers.items():
            poly = PolynomialFeatures(degree=3, interaction_only=True)
            X_train_poly = poly.fit_transform(X_train_selected)
            X_test_poly = poly.transform(X_test_selected)
            clf.fit(X_train_poly, y_train)

            y_pred = clf.predict(X_test_poly)
            y_prob = clf.predict_proba(X_test_poly) if hasattr(clf, "predict_proba") else None

            eval_result = evaluatelog_result(y_test, y_pred, clf_name, logger, y_prob)
            eval_result["Scaler"] = scaler_name
            eval_result["FeatureSelection"] = selector_name
            eval_result["Classifier"] = clf_name
            results.append(eval_result)

results_df = pd.DataFrame(results)

results_df.to_excel("no_kfold_poly.xlsx", index=False)